<a href="https://colab.research.google.com/github/Thomas-George-T/Retro-Reel-Recommender/blob/main/Retro_Reel_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrol Reel Recommender

## Data Acquisition

In [369]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [370]:
# display full data in the cell without truncating
pd.set_option('display.max_colwidth', None)

In [371]:
!wget -q -O 'retro.json' https://prosona.b-cdn.net/Movies/catalog-v2.txt
print("Data Downloaded!")

Data Downloaded!


In [372]:
retro_raw = pd.read_json('retro.json')
retro_raw.head()

,Movies
0,"{'ID': 'b70eec12b79148dfbc97ac3b69ca49bf', 'Title': 'Angel On My Shoulder', 'ReleaseYear': 1946, 'Genres': ['Adventure', 'Comedy', 'Fantasy'], 'Synopsis': 'The Devil arranges for a deceased gangster to return to Earth as a well-respected judge to make up for his previous life.', 'Directors': ['Archie Mayo'], 'Cast': ['Paul Muni', 'Anne Baxter', 'Claude Rains'], 'Trivia': ['Director Archie Mayo clashed repeatedly with star Paul Muni during production. Muni didn't make another film for eight years.', 'The film's original title was ""Me and Satan"" but producer Charles R. Rogers decided to change it to Angel on My Shoulder (1946) on the conclusion that no one would want to go to see a film about the Devil.'], 'Keywords': 'Devil Soul angel on my shoulder 1946 adventure comedy fantasy archie mayo paul muni anne baxter claude rains', 'TrailerLink': 'https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/Angel%20On%20My%20Shoulder%20%281946%29%20Trailer.mp4', 'MovieLink': 'https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/Angel%20On%20My%20Shoulder%20%281946%29.mp4', 'HorizontalPosterLink': 'https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/horizontal.jpg', 'VerticalPosterLink': 'https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/vertical.jpg', 'SnapshotLink': 'https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/screenshot.jpg', 'SubtitlesLink': '', 'Offset': 0.0}"
1,"{'ID': '1950e4a11304438dbfd4792a908fc76f', 'Title': 'Flirting with Fate', 'ReleaseYear': 1916, 'Genres': ['Adventure', 'Comedy'], 'Synopsis': 'In the midst of an emotional depression, a man hires a murderer to kill him. But the despair soon passes, and the man must now escape the killer he's hired to end his life.', 'Directors': ['Christy Cabanne'], 'Cast': ['Douglas Fairbanks', 'Jewel Carmen', 'Howard Gaye'], 'Trivia': [], 'Keywords': 'hitman assasin killer escape flirting with fate 1916 adventure comedy christy cabanne douglas fairbanks jewel carmen howard gaye', 'TrailerLink': 'https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/preview.mp4', 'MovieLink': 'https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/movie.mp4', 'HorizontalPosterLink': 'https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/horizontal.png', 'VerticalPosterLink': 'https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/vertical.png', 'SnapshotLink': 'https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/screenshot.jpg', 'SubtitlesLink': '', 'Offset': 0.0}"
2,"{'ID': '37d8cae8d87f4dbc9cf8967454537afb', 'Title': 'Detour', 'ReleaseYear': 1945, 'Genres': ['Crime', 'Drama', 'Noir'], 'Synopsis': 'Chance events trap hitch-hiker Al Roberts in a tightening net of film noir trouble.', 'Directors': ['Edgar G. Ulmer'], 'Cast': ['Tom Neal', 'Ann Savage', 'Claudia Drake'], 'Trivia': ['The budget PRC gave director Edgar G. Ulmer for this film was so small that the 1941 Lincoln Continental V-12 convertible driven by Charles Haskell was actually Ulmer's personal car.', 'Ann Savage worked on her biography for the last decade of her life. It was released in early 2010 called ""Savage Detours.""', 'Shot quickly in mostly two locations: the hotel room apartment, and the car in front of a rear projection screen on a soundstage at PRC.'], 'Keywords': 'detour 1945 crime drama noir edgar g. ulmer tom neal ann savage claudia drake', 'TrailerLink': 'https://prosona.b-cdn.net/Movies/Detour%20-%201945/Detour%20-%201945%20Trailer.mp4', 'MovieLink': 'https://prosona.b-cdn.net/Movies/Detour%20-%201945/Detour-1945.mp4', 'HorizontalPosterLink': 'https://prosona.b-cdn.net/Movies/Detour%20-%201945/horizontal-poster.jpg', 'VerticalPosterLink': 'https://prosona.b-cdn.net/Movies/Detour%20-%201945/vertical-poster.jpg', 'SnapshotLink': 'https://prosona.b-cdn.net/Movies/Detour%20-%201945/snapshot.jpg', 'SubtitlesLink': 'https://prosona.b-cdn.net

## Data Preprocessing

In [373]:
retro_raw_flatten = pd.json_normalize(retro_raw['Movies'])
retro_raw_flatten.head()

,ID,Title,ReleaseYear,Genres,Synopsis,Directors,Cast,Trivia,Keywords,TrailerLink,MovieLink,HorizontalPosterLink,VerticalPosterLink,SnapshotLink,SubtitlesLink,Offset
0,b70eec12b79148dfbc97ac3b69ca49bf,Angel On My Shoulder,1946,"[Adventure, Comedy, Fantasy]",The Devil arranges for a deceased gangster to return to Earth as a well-respected judge to make up for his previous life.,[Archie Mayo],"[Paul Muni, Anne Baxter, Claude Rains]","[Director Archie Mayo clashed repeatedly with star Paul Muni during production. Muni didn't make another film for eight years., The film's original title was ""Me and Satan"" but producer Charles R. Rogers decided to change it to Angel on My Shoulder (1946) on the conclusion that no one would want to go to see a film about the Devil.]",Devil Soul angel on my shoulder 1946 adventure comedy fantasy archie mayo paul muni anne baxter claude rains,https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/Angel%20On%20My%20Shoulder%20%281946%29%20Trailer.mp4,https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/Angel%20On%20My%20Shoulder%20%281946%29.mp4,https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/horizontal.jpg,https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/vertical.jpg,https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/screenshot.jpg,,0.0
1,1950e4a11304438dbfd4792a908fc76f,Flirting with Fate,1916,"[Adventure, Comedy]","In the midst of an emotional depression, a man hires a murderer to kill him. But the despair soon passes, and the man must now escape the killer he's hired to end his life.",[Christy Cabanne],"[Douglas Fairbanks, Jewel Carmen, Howard Gaye]",[],hitman assasin killer escape flirting with fate 1916 adventure comedy christy cabanne douglas fairbanks jewel carmen howard gaye,https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/preview.mp4,https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/movie.mp4,https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/horizontal.png,https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/vertical.png,https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/screenshot.jpg,,0.0
2,37d8cae8d87f4dbc9cf8967454537afb,Detour,1945,"[Crime, Drama, Noir]",Chance events trap hitch-hiker Al Roberts in a tightening net of film noir trouble.,[Edgar G. Ulmer],"[Tom Neal, Ann Savage, Claudia Drake]","[The budget PRC gave director Edgar G. Ulmer for this film was so small that the 1941 Lincoln Continental V-12 convertible driven by Charles Haskell was actually Ulmer's personal car., Ann Savage worked on her biography for the last decade of her life. It was released in early 2010 called ""Savage Detours."", Shot quickly in mostly two locations: the hotel room apartment, and the car in front of a rear projection screen on a soundstage at PRC.]",detour 1945 crime drama noir edgar g. ulmer tom neal ann savage claudia drake,https://prosona.b-cdn.net/Movies/Detour%20-%201945/Detour%20-%201945%20Trailer.mp4,https://prosona.b-cdn.net/Movies/Detour%20-%201945/Detour-1945.mp4,https://prosona.b-cdn.net/Movies/Detour%20-%201945/horizontal-poster.jpg,https://prosona.b-cdn.net/Movies/Detour%20-%201945/vertical-poster.jpg,https://prosona.b-cdn.net/Movies/Detour%20-%201945/snapshot.jpg,https://prosona.b-cdn.net/Movies/Detour%20-%201945/sub%20EN.srt,0.0
3,b99dd03c00cc4fb9b2d4ad26082de498,The Big Trees,1952,"[Action, Romance, Western]",A Quaker colony tries to save the giant sequoias from a timber baron.,[Felix E. Feist],"[Kirk Douglas, Eve Miller, Patrice Wymore]","[According to Kirk Douglas in his autobiography ""The Ragman's Son"", he agreed to act in this film for free, in order to end his contract with Warner Bros., The reading that Fallon does at the dinner is from Psalms 41, verse 1., In the same year, Kirk Douglas starred in a movie titled The Big Trees and a movie titled The Big Sky. Though they were narra

In [374]:
retro_raw_flatten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    49 non-null     object 
 1   Title                 49 non-null     object 
 2   ReleaseYear           49 non-null     int64  
 3   Genres                49 non-null     object 
 4   Synopsis              49 non-null     object 
 5   Directors             49 non-null     object 
 6   Cast                  49 non-null     object 
 7   Trivia                49 non-null     object 
 8   Keywords              49 non-null     object 
 9   TrailerLink           49 non-null     object 
 10  MovieLink             49 non-null     object 
 11  HorizontalPosterLink  49 non-null     object 
 12  VerticalPosterLink    49 non-null     object 
 13  SnapshotLink          49 non-null     object 
 14  SubtitlesLink         49 non-null     object 
 15  Offset                49 

## Feature Selection

We chose ID, Title, ReleaseYear, Genres, Synopsis, Directors and Cast as our features. They are to generate a bag of keywords which we will use as our basis of the cosimilarity matrix. 

In [375]:
df4 = retro_raw_flatten [['ID','Title','ReleaseYear','Genres','Synopsis','Directors','Cast']]
df4

,ID,Title,ReleaseYear,Genres,Synopsis,Directors,Cast
0,b70eec12b79148dfbc97ac3b69ca49bf,Angel On My Shoulder,1946,"[Adventure, Comedy, Fantasy]",The Devil arranges for a deceased gangster to return to Earth as a well-respected judge to make up for his previous life.,[Archie Mayo],"[Paul Muni, Anne Baxter, Claude Rains]"
1,1950e4a11304438dbfd4792a908fc76f,Flirting with Fate,1916,"[Adventure, Comedy]","In the midst of an emotional depression, a man hires a murderer to kill him. But the despair soon passes, and the man must now escape the killer he's hired to end his life.",[Christy Cabanne],"[Douglas Fairbanks, Jewel Carmen, Howard Gaye]"
2,37d8cae8d87f4dbc9cf8967454537afb,Detour,1945,"[Crime, Drama, Noir]",Chance events trap hitch-hiker Al Roberts in a tightening net of film noir trouble.,[Edgar G. Ulmer],"[Tom Neal, Ann Savage, Claudia Drake]"
3,b99dd03c00cc4fb9b2d4ad26082de498,The Big Trees,1952,"[Action, Romance, Western]",A Quaker colony tries to save the giant sequoias from a timber baron.,[Felix E. Feist],"[Kirk Douglas, Eve Miller, Patrice Wymore]"
4,5566ce75b5954f3dad323c2f154856a3,The Jungle Book,1942,"[Action, Adventure]",\r\nA boy raised by wild animals tries to adapt to human village life.,[Zoltan Korda],"[Sabu, Joseph Calleia, John Qualen]"
5,62e08996bb034303a417d10e136bedda,The Lost World,1925,"[Action, Fantasy, Thriller]",The first film adaptation of Sir Arthur Conan Doyle's classic novel about a land where prehistoric creatures still roam.,[Harry O. Hoyt],"[Wallace Beery, Bessie Love, Lloyd Hughes]"
6,74ac96777a4d404b8eba028e44f8f06b,Suddenly,1954,"[Crime, Drama, Noir]","In the city of Suddenly, three gangsters trap the Benson family in their own house, on the top of a hill nearby the railroad station, with the intention of killing the president of the USA.",[],[]
7,4014bc52406d47d88f077ab9ca6e988c,Bloody Pit Of Horror,1965,"[Horror, Thriller]","A photographer and his models go to an old, abandoned castle to shoot some sexy covers for horror novels. Unbeknownst to them, the castle is inhabited by a lunatic who believes himself to be the reincarnated spirit of a 17th-century executioner whose job it is to protect the castle against intruders.",[Domenico Massimo],"[Mickey Hargitay, Walter Brandi, Luisa Baratto]"
8,97063c49fbe94966a7f883594896855b,Charlie Chaplin's The Rink,1916,"[Short, Comedy]","After causing restaurant chaos at work, a bumbling waiter tears up the local roller rink with his skating.",[Charles Chaplin],"[Charles Chaplin, Edna Purviance, James T. Kelley]"
9,3cb4bd7eed36485c8faf54c41c243f20,Texas Terror,1935,"[Action, Adventure, Drama]","Sheriff John Higgins quits and goes into prospecting after he thinks he has killed his best friend in shooting it out with robbers. He encounters his dead buddy's daughter, who has come from back east, and helps her run her ranch. Then she finds out about his past.",[Robert N. Bradbury],"[John Wayne, Lucile Browne, LeRoy Mason]"


In [376]:
df4['Synopsis']

0                                                                                                                                                                                                                                                 The Devil arranges for a deceased gangster to return to Earth as a well-respected judge to make up for his previous life.
1                                                                                                                                                                                              In the midst of an emotional depression, a man hires a murderer to kill him. But the despair soon passes, and the man must now escape the killer he's hired to end his life.
2                                                                                                                                                                                                                                                                               

## Data Processing

Using regular expression library, remove special characters and punctuations from the columns and make them into lower case while at it. This is so as to generate the keywords more easily. We remove the space between the names to avoid the scenario in the cosimilarity index where the cast/director have the same first name but different last name and the similarity index gets skewed.


In [377]:
import re

#set up punctuations to be replaced
remove_punct_space = re.compile("(\.)|(\')|(\s)")
remove_punct = re.compile("(\.)|(\')|(\:)|(\,)|(\-)|(\$)|(\")")

In [378]:
df4.iloc[4,4] = df4.iloc[4,4].replace('\r\n','')
df4['Directors'] = df4['Directors'].map(lambda x : [ remove_punct_space.sub("", y.lower()) for y in x ])
df4['Cast'] = df4['Cast'].map(lambda x : [ remove_punct_space.sub("", y.lower()) for y in x ])

df4['Synopsis'] = df4['Synopsis'].map(lambda x: remove_punct.sub("", x.lower()))
df4['Title'] = df4['Title'].map(lambda x: remove_punct.sub("", x.lower()))

# Make the rest of the columns also lower case
df4['Genres'] = df4['Genres'].map(lambda x: [ y.lower() for y in x ])
df4

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

,ID,Title,ReleaseYear,Genres,Synopsis,Directors,Cast
0,b70eec12b79148dfbc97ac3b69ca49bf,angel on my shoulder,1946,"[adventure, comedy, fantasy]",the devil arranges for a deceased gangster to return to earth as a wellrespected judge to make up for his previous life,[archiemayo],"[paulmuni, annebaxter, clauderains]"
1,1950e4a11304438dbfd4792a908fc76f,flirting with fate,1916,"[adventure, comedy]",in the midst of an emotional depression a man hires a murderer to kill him but the despair soon passes and the man must now escape the killer hes hired to end his life,[christycabanne],"[douglasfairbanks, jewelcarmen, howardgaye]"
2,37d8cae8d87f4dbc9cf8967454537afb,detour,1945,"[crime, drama, noir]",chance events trap hitchhiker al roberts in a tightening net of film noir trouble,[edgargulmer],"[tomneal, annsavage, claudiadrake]"
3,b99dd03c00cc4fb9b2d4ad26082de498,the big trees,1952,"[action, romance, western]",a quaker colony tries to save the giant sequoias from a timber baron,[felixefeist],"[kirkdouglas, evemiller, patricewymore]"
4,5566ce75b5954f3dad323c2f154856a3,the jungle book,1942,"[action, adventure]",a boy raised by wild animals tries to adapt to human village life,[zoltankorda],"[sabu, josephcalleia, johnqualen]"
5,62e08996bb034303a417d10e136bedda,the lost world,1925,"[action, fantasy, thriller]",the first film adaptation of sir arthur conan doyles classic novel about a land where prehistoric creatures still roam,[harryohoyt],"[wallacebeery, bessielove, lloydhughes]"
6,74ac96777a4d404b8eba028e44f8f06b,suddenly,1954,"[crime, drama, noir]",in the city of suddenly three gangsters trap the benson family in their own house on the top of a hill nearby the railroad station with the intention of killing the president of the usa,[],[]
7,4014bc52406d47d88f077ab9ca6e988c,bloody pit of horror,1965,"[horror, thriller]",a photographer and his models go to an old abandoned castle to shoot some sexy covers for horror novels unbeknownst to them the castle is inhabited by a lunatic who believes himself to be the reincarnated spirit of a 17thcentury executioner whose job it is to protect the castle against intruders,[domenicomassimo],"[mickeyhargitay, walterbrandi, luisabaratto]"
8,97063c49fbe94966a7f883594896855b,charlie chaplins the rink,1916,"[short, comedy]",after causing restaurant chaos at work a bumbling waiter tears up the local roller rink with his skating,[charleschaplin],"[charleschaplin, ednapurviance, jamestkelley]"
9,3cb4bd7eed36485c8faf54c41c243f20,texas terror,1935,"[action, adventure, drama]",sheriff john higgins quits and goes into prospecting after he thinks he has killed his best friend in shooting it out with robbers he encounters his dead buddys daughter who has come from back east and helps her run her ranch then she finds out about his past,[robertnbradbury],"[johnwayne, lucilebrowne, leroymason]"


## NLP
We will generate keywords from the synopsis column using rake from nltk. Ultimately, we will use all the columns to generate the bag of words.

In [379]:
!pip install nltk
from nltk.corpus import stopwords 

In [380]:
!pip install rake-nltk
from rake_nltk import Rake

In [381]:
# initializing the new column
df4['Key_words'] = ""

for index, row in df4.iterrows():
    synopsis = row['Synopsis']
    
    # instantiating Rake, by default it uses english stopwords from NLTK and discards all puntuation
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(synopsis)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()

    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())
    df4['Key_words'][index] = row['Key_words']
       

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, se

In [382]:
df4.drop(columns=['ID','Synopsis'], inplace=True)
df4.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Title,ReleaseYear,Genres,Directors,Cast,Key_words
0,angel on my shoulder,1946,"[adventure, comedy, fantasy]",[archiemayo],"[paulmuni, annebaxter, clauderains]","[earth, return, wellrespected, judge, deceased, gangster, previous, life, devil, arranges, make]"
1,flirting with fate,1916,"[adventure, comedy]",[christycabanne],"[douglasfairbanks, jewelcarmen, howardgaye]","[life, midst, man, hires, killer, hes, hired, emotional, depression, murderer, despair, soon, passes, kill, escape, must, end]"
2,detour,1945,"[crime, drama, noir]",[edgargulmer],"[tomneal, annsavage, claudiadrake]","[tightening, net, film, noir, trouble, chance, events, trap, hitchhiker, al, roberts]"
3,the big trees,1952,"[action, romance, western]",[felixefeist],"[kirkdouglas, evemiller, patricewymore]","[timber, baron, quaker, colony, tries, giant, sequoias, save]"
4,the jungle book,1942,"[action, adventure]",[zoltankorda],"[sabu, josephcalleia, johnqualen]","[human, village, life, adapt, wild, animals, tries, boy, raised]"


In [383]:
df4['ReleaseYear'] = df4['ReleaseYear'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [384]:
df4['bag_of_words'] = ''
columns = df4.columns
for index, row in df4.iterrows():
    words = ''
    for col in columns:
        if col != 'ReleaseYear' and col != 'Title':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df4['bag_of_words'].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0                              angel on my shoulder 1946 adventure comedy fantasy archiemayo paulmuni annebaxter clauderains earth return wellrespected judge deceased gangster previous life devil arranges make  
1    flirting with fate 1916 adventure comedy christycabanne douglasfairbanks jewelcarmen howardgaye life midst man hires killer hes hired emotional depression murderer despair soon passes kill escape must end  
2                                                               detour 1945 crime drama noir edgargulmer tomneal annsavage claudiadrake tightening net film noir trouble chance events trap hitchhiker al roberts  
3                                                                  the big trees 1952 action romance western felixefeist kirkdouglas evemiller patricewymore timber baron quaker colony tries giant sequoias save  
4                                                                          the jungle book 1942 action adventure zoltankorda sabu josephcalleia johnqual



---



In [385]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df4['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(df4.Title)
indices

0                       angel on my shoulder
1                         flirting with fate
2                                     detour
3                              the big trees
4                            the jungle book
5                             the lost world
6                                   suddenly
7                       bloody pit of horror
8                  charlie chaplins the rink
9                               texas terror
10                             the iron mask
11                              barefoot boy
12                                borderline
13             charlie chaplins the vagabond
14                           galloping romeo
15                           his girl friday
16                           the star packer
17                            scarlet street
18    charlie chaplins a burlesque on carmen
19             charlie chaplins the pawnshop
20                                   manfish
21                         carnival of souls
22        

In [386]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.12038585, 0.        , ..., 0.08891084, 0.04012862,
        0.03940552],
       [0.12038585, 1.        , 0.        , ..., 0.12309149, 0.03703704,
        0.0727393 ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.04029115],
       ...,
       [0.08891084, 0.12309149, 0.        , ..., 1.        , 0.0410305 ,
        0.04029115],
       [0.04012862, 0.03703704, 0.        , ..., 0.0410305 , 1.        ,
        0.03636965],
       [0.03940552, 0.0727393 , 0.04029115, ..., 0.04029115, 0.03636965,
        1.        ]])

new logic


In [387]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
   
    recommended_movies = []
    
    # getting the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df4.Title)[i])
        #recommended_movies.append(list(rr.Title)[i])       
      
    return recommended_movies

In [388]:
recommendations('angel on my shoulder')

['flirting with fate',
 'the jungle book',
 'life with father',
 'my man godfrey',
 'charlie chaplins a burlesque on carmen',
 'the snows of kilimanjaro',
 'charlie chaplins the vagabond',
 'millie',
 'manfish',
 'penny serenade']

OLD keywords logic

recommendations('Angel On My Shoulder')

```
['My Man Godfrey',
 'Flirting with Fate',
 "Charlie Chaplin's A Burlesque On Carmen",
 'Texas Terror',
 'Million Dollar Kid',
 'Life With Father',
 'Barefoot Boy',
 'Galloping Romeo',
 'Manfish',
 'The Outlaw']
```





experimental : change `idx = indices[indices == title].index[0]` into `idx = retro_raw_flatten[retro_raw_flatten == title].index[0]`



---



## What if we used the Title of the Movie instead?

In [389]:
# instantiating and generating the count matrix
count1 = CountVectorizer()
count_matrix1 = count1.fit_transform(df4['Title'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices1 = pd.Series(df4.Title)
indices1

0                       angel on my shoulder
1                         flirting with fate
2                                     detour
3                              the big trees
4                            the jungle book
5                             the lost world
6                                   suddenly
7                       bloody pit of horror
8                  charlie chaplins the rink
9                               texas terror
10                             the iron mask
11                              barefoot boy
12                                borderline
13             charlie chaplins the vagabond
14                           galloping romeo
15                           his girl friday
16                           the star packer
17                            scarlet street
18    charlie chaplins a burlesque on carmen
19             charlie chaplins the pawnshop
20                                   manfish
21                         carnival of souls
22        

In [390]:
# generating the cosine similarity matrix
cosine_sim1 = cosine_similarity(count_matrix1, count_matrix1)
cosine_sim1

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.33333333, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.33333333, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [391]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations1(title, cosine_sim1 = cosine_sim1):
   
    recommended_movies1 = []
    
    # gettin the index of the movie that matches the title
    idx1 = indices1[indices1 == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series1 = pd.Series(cosine_sim1[idx1]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_5_indexes1 = list(score_series1.iloc[1:6].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_5_indexes1:
        recommended_movies1.append(list(retro_raw_flatten.Title)[i])
        
    return recommended_movies1

In [392]:
recommendations1("charlie chaplins the rink")

["Charlie Chaplin's The Pawnshop",
 "Charlie Chaplin's The Vagabond",
 "Charlie Chaplin's The Knockout",
 "Charlie Chaplin's A Fair Exchange",
 "Charlie Chaplin's A Burlesque On Carmen"]

recommendations1("Charlie Chaplin's The Rink")

```
["Charlie Chaplin's The Pawnshop",
 "Charlie Chaplin's The Vagabond",
 "Charlie Chaplin's The Knockout",
 "Charlie Chaplin's A Fair Exchange",
 "Charlie Chaplin's A Burlesque On Carmen"]
```



# Combining both the approaches.
Also adding NLP to increase accuracy.

In [393]:
# instantiating and generating the count matrix
count3 = CountVectorizer()
count_matrix3 = count3.fit_transform(df4['bag_of_words'])
count4 = CountVectorizer()
count_matrix4 = count4.fit_transform(df4['Title'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices3 = pd.Series(df4.Title)
indices4 = pd.Series(df4.Title)

In [394]:
# generating the cosine similarity matrix
cosine_sim3 = cosine_similarity(count_matrix3, count_matrix3)

# generating the cosine similarity matrix
cosine_sim4 = cosine_similarity(count_matrix4, count_matrix4)


In [439]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations3(title, cosine_sim3 = cosine_sim3, cosine_sim4 = cosine_sim4):
   
    recommended_movies3 = []
    recommended_movies4 = []
    final_mov = []
    
    # gettin the index of the movie that matches the title
    idx3 = indices3[indices3 == title].index[0]
    idx4 = indices4[indices4 == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series3 = pd.Series(cosine_sim3[idx3]).sort_values(ascending = False)
    score_series4 = pd.Series(cosine_sim4[idx4]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes3 = list(score_series3.iloc[1:11].index)
    top_10_indexes4 = list(score_series4.iloc[1:11].index)

    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes3:
        recommended_movies3.append(list(df4.Title)[i])
    
    for i in top_10_indexes4:
        recommended_movies4.append(list(df4.Title)[i])

    final_mov =  list(recommended_movies3) + list(recommended_movies4)

    return list(set(final_mov))

In [443]:
recommendations3("angel on my shoulder")

['scarlet street',
 'the star packer',
 'the snows of kilimanjaro',
 'charlie chaplins the vagabond',
 'barefoot boy',
 'millie',
 'house on haunted hill',
 'his girl friday',
 'the jungle book',
 'charlie chaplins a burlesque on carmen',
 'manfish',
 'my man godfrey',
 'penny serenade',
 'charlie chaplins the pawnshop',
 'life with father',
 'flirting with fate']